In [1]:
print('Importing libraries (~10sec)...')

import sys
print(sys.version)
import os
import os.path
import random
import re
import heapq
from importlib import reload

import numpy as np
import pandas as pd

from nltk import tokenize

#from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Activation, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.metrics import top_k_categorical_accuracy

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
except:
    plt = None

# local modules
import data as datamodule
import util

Importing libraries (~10sec)...
3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abdullahkhilji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
debug            = 0         # 0 or 1
DATASET          = 'gutenbergs' # dataset to use (gutenbergs, alice1)
TRAIN_AMOUNT     = 0.9      # percent of training data to use (for debugging), 0.0 to 1.0
NEPOCHS          = 100         # number of epochs to train model
LAYERS           = 3         # number of RNN layers, 1 to 3
DROPOUT          = 0.1       # amount of dropout to apply after each layer, 0.0 to 1.0
NVOCAB           = 715   # number of vocabulary words to use
EMBEDDING_DIM    = 50       # dimension of embedding layer - 50, 100, 200, 300
TRAINABLE        = True      # allow embedding matrix to be trained?
NHIDDEN1        = 200# size of hidden layer(s)
NHIDDEN2        = 100
NHIDDEN         = 100
N            = 10        # amount to unfold recurrent network
RNN_CLASS        = GRU       # type of RNN to use - SimpleRNN, LSTM, or GRU
BATCH_SIZE       = 20000        # size of batch to use for training
INITIAL_EPOCH    = 0         # to continue training
PATIENCE         = 10        # stop after this many epochs of no improvement
VALIDATION_SPLIT = 0.01      # percent of training data to use for validation (0.01 ~10k tokens)
NTEST            = 10000     # number of tokens to use for testing
OPTIMIZER        = 'adam'    # optimizing algorithm to use (sgd, rmsprop, adam, adagrad, adadelta, adamax, nadam)
INITIALIZER      = 'uniform' # random weight initializer (uniform, normal, lecun_uniform, glorot_uniform [default])
SEED             = 0         # random number seed

# LOSS_FN    = 'categorical_crossentropy' # allows calculation of top_k_accuracy, but requires one-hot encoding y values
LOSS_FN    = 'sparse_categorical_crossentropy'
BASE_DIR   = '.'
GLOVE_DIR  = BASE_DIR + '/_vectors/glove.6B'
GLOVE_FILE = '/Users/abdullahkhilji/Extras/DeepDive/Final to check/tf/Word-Prediction-with-LSTM/vectors/glove.6B/vectors.txt '
MODEL_DIR  = BASE_DIR + '/_models/' + DATASET
MODEL_FILE = MODEL_DIR + "/model-train_amount-%s-nvocab-%d-embedding_dim-%d-nhidden-%d-n-%d.h5" % \
                         (TRAIN_AMOUNT, NVOCAB, EMBEDDING_DIM, NHIDDEN, N)

In [3]:

import util
data = datamodule.Data(DATASET)
data.folder = '/Users/abdullahkhilji/Extras/DeepDive/Final to check/tf/Word-Prediction-with-LSTM/data/' + data.dataset

In [4]:
import data as datamodule
data.prepare(nvocab=NVOCAB)

Reading texts...
/Users/abdullahkhilji/Extras/DeepDive/Final to check/tf/Word-Prediction-with-LSTM/data/gutenbergs/testfile.txt
Split into paragraphs, shuffle, recombine...
Tokenizing text (~15sec)...
Find vocabulary words...
Convert text to numeric sequence, skipping OOV words...


In [5]:
x_train, y_train, x_test, y_test = data.split(n=N, ntest=NTEST,
                                              train_amount=TRAIN_AMOUNT, debug=debug)

Create train and test sets...


In [6]:
len(y_train)

169171

In [7]:
print('Reading word vectors (~15sec)...')
word_vectors = {}
with open('/Users/abdullahkhilji/Extras/DeepDive/Final to check/tf/Word-Prediction-with-LSTM/vectors/a0.1', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_vectors[word] = coefs

Reading word vectors (~15sec)...


In [8]:
def get_embedding_matrix(data, word_vectors, nvocab):
    nwords = nvocab
    embedding_dim = len(word_vectors['a'])
    E = np.zeros((nwords + 1, embedding_dim))
    for word, iword in data.word_to_iword.items():
        if iword > nvocab:
            continue
        word_vector = word_vectors.get(word)
        # words not found in embedding index will be all zeros
        if word_vector is not None:
            E[iword] = word_vector
    return E
E = get_embedding_matrix(data, word_vectors, NVOCAB)

In [9]:
print('number of word vectors in matrix E',len(E))
print('example word vector:',E[1])

number of word vectors in matrix E 716
example word vector: [-0.0343038   0.30809    -0.0266542  -0.91010702 -0.31334999  0.0252021
  0.81171     0.51103002  0.26535901  0.0704833  -0.024386   -0.31169701
 -0.31038001  0.59833902 -0.24935099  0.454761    0.345485   -0.546965
  0.515136    0.56245202  0.50753498  0.065713    0.75871301  0.52148998
 -0.16196799 -0.57184201 -0.19863699 -0.73529899 -0.23457199 -0.25543699
  0.742603    0.49605101 -0.67958701  0.358735   -0.47995099 -0.37514001
  0.58205098 -0.476145    0.0480055  -0.67645198 -0.407617   -0.44757599
  0.509642   -0.661264   -0.62075698 -0.463929   -0.69187403 -0.71164799
 -0.63205701  0.241734  ]


In [10]:
import tensorflow as tf
embedding_layer = Embedding(input_dim=NVOCAB+1, output_dim=EMBEDDING_DIM,
                            input_length=N-1, weights=[E])


from keras.layers import Dense, Activation, Flatten
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(NHIDDEN1, return_sequences=True))
model.add(LSTM(NHIDDEN2, return_sequences=True))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(NVOCAB))
model.add(Activation(tf.nn.softmax))
metrics = ['accuracy'] # loss is always the first metric returned from the fit method
model.compile(loss=LOSS_FN, optimizer=OPTIMIZER, metrics=metrics)
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, nb_epoch=NEPOCHS,
                        validation_split=VALIDATION_SPLIT,)

util.uprint('Final epoch generated text:', util.generate_text(model, data, N))
print()

print('history')
print(history.history)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 167479 samples, validate on 1692 samples
Epoch 1/100
167479/167479 [==============================] - 69s 411us/step - loss: 5.6652 - acc: 0.2479 - val_loss: 3.3879 - val_acc: 0.2677
Epoch 2/100
167479/167479 [==============================] - 67s 397us/step - loss: 3.5465 - acc: 0.2853 - val_loss: 3.3422 - val_acc: 0.2677
Epoch 3/100
167479/167479 [==============================] - 68s 408us/step - loss: 3.4049 - acc: 0.2878 - val_loss: 3.2308 - val_acc: 0.2677
Epoch 4/100
167479/167479 [==============================] - 66s 393us/step - loss: 3.3458 - acc: 0.2878 - val_loss: 3.2342 - val_acc: 0.2677
Epoch 5/100
167479/167479 [==============================] - 66s 392us/step - loss: 3.3256 - acc: 0.2878 - val_loss: 3.2056 - val_acc: 0.2677
Epoch 6/100
167479/167479 [==============================] - 66s 394us/step - loss: 3.3155 - acc: 0.2878 - val_loss: 3.2046 - val_acc: 0.2677
Epoch 7/100
167479/167479 [==============================] - 72s 428us/step - loss: 3.3091 - acc: 

167479/167479 [==============================] - 66s 395us/step - loss: 1.9651 - acc: 0.5207 - val_loss: 1.9896 - val_acc: 0.5745
Epoch 59/100
167479/167479 [==============================] - 66s 395us/step - loss: 1.9599 - acc: 0.5214 - val_loss: 2.0065 - val_acc: 0.5691
Epoch 60/100
167479/167479 [==============================] - 67s 398us/step - loss: 1.9547 - acc: 0.5220 - val_loss: 1.9788 - val_acc: 0.5745
Epoch 61/100
167479/167479 [==============================] - 67s 398us/step - loss: 1.9483 - acc: 0.5228 - val_loss: 1.9571 - val_acc: 0.5745
Epoch 62/100
167479/167479 [==============================] - 68s 409us/step - loss: 1.9440 - acc: 0.5234 - val_loss: 1.9637 - val_acc: 0.5745
Epoch 63/100
167479/167479 [==============================] - 66s 395us/step - loss: 1.9373 - acc: 0.5248 - val_loss: 1.9816 - val_acc: 0.5697
Epoch 64/100
167479/167479 [==============================] - 67s 397us/step - loss: 1.9308 - acc: 0.5259 - val_loss: 1.9721 - val_acc: 0.5721
Epoch 65/100

In [11]:
print('generated text:')
nsentences = 10
nwords_to_generate = 20
k = 3
for i in range(nsentences):
    util.uprint(util.generate_text(model, data, N, nwords_to_generate, k))

# calculate test accuracy on the heldout test data
loss, accuracy = model.evaluate(x_test, y_test, BATCH_SIZE, verbose=0)
print("Test loss:",loss)
print("Test accuracy:",accuracy)

generated text:
ls ls parameter cd ls ls parameter cd ls cd filename ls ls filename ls cd filename ls cd ls
filename ls cd filename ls cd ls cd ls ls cd cd ls cd filename ls emacs cd cd ls
cd sfn sfn parameter emacs filename emacs filename ls parameter cd filename ls cd cd cd ls cd ls parameter
average average filename basic fg basic basic average filename fg cd filename fg fg fg filename cd cd filename ls
average parameter average filename cc parameter testrand fg average parameter cd filename fg fg cc parameter average filename sim fg
parameter parameter more '' filename ls more parameter cd filename more cd filename ls more filename more parameter more filename
fg f f filename tip filename logout f tip f f filename mail tip tip logout who who f f
filename emacs filename cd ls emacs filename cd cd filename emacs emacs emacs emacs roff filename roff emacs filename cd
parameter emacs cc average parameter cc filename emacs cc parameter /user/grads/xxxxxxx/connect/bin/makesim parameter e